In [1]:
import csv
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from random import randint
import urllib.parse


website = """
#########################################
#  WEBSITE: naukri.com #
######################################### 
"""
print(website)
start_time = datetime.now()
print('Crawl starting time : {}'.format(start_time.time()))
print()

# Prompt the user to enter a job role
job_role = input("Enter the job role you want to search for: ")

# CSV file setup
csv_file = open('job_listings.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Job Title', 'Company Name', 'Rating', 'Experience', 'Location', 'Salary', 'Minimum Requirements', 'Skills', 'Posted Date'])

def generate_url(index, job_role):
    encoded_role = urllib.parse.quote(job_role)  # URL encode the job role
    if index == 1:
        return f"https://www.naukri.com/{encoded_role}-jobs?k={encoded_role}&nignbevent_src=jobsearchDeskGNB"
    else:
        return f"https://www.naukri.com/{encoded_role}-jobs?k={encoded_role}&nignbevent_src=jobsearchDeskGNB&page={index}"

def extract_rating(rating_a):
    if rating_a is None or rating_a.find('span', class_="main-2") is None:
        return "None"
    else:
        return rating_a.find('span', class_="main-2").text

def parse_job_data_from_soup(page_jobs):
    print("********PAGE_JOBS***********")
    for job in page_jobs:
        job = BeautifulSoup(str(job), 'html.parser')
        row1 = job.find('div', class_="row1")
        row2 = job.find('div', class_="row2")
        row3 = job.find('div', class_="row3")
        row4 = job.find('div', class_="row4")
        row5 = job.find('div', class_="row5")
        row6 = job.find('div', class_="row6")

        job_title = row1.a.text if row1 and row1.a else "Title not found"
        company_name = row2.span.a.text if row2 and row2.span and row2.span.a else "Company not found"
        rating_a = row2.span if row2 else None
        rating = extract_rating(rating_a)

        job_details = row3.find('div', class_="job-details") if row3 else None
        ex_wrap = job_details.find('span', class_="exp-wrap").span.span.text if job_details and job_details.find('span', class_="exp-wrap") else "Experience not found"
        location = job_details.find('span', class_="loc-wrap ver-line").span.span.text if job_details and job_details.find('span', class_="loc-wrap ver-line") else "Location not found"

        salary_div = job.find('span', class_='sal-wrap')
        salary = salary_div.text.strip() if salary_div else "Salary not found"

        min_requirements = row4.span.text if row4 and row4.span else "Requirements not found"
        skills = row5.text.strip() if row5 else "Skills not found"
        posted_date = row6.text.strip() if row6 else "Date not found"

        csv_writer.writerow([job_title, company_name, rating, ex_wrap, location, salary, min_requirements, skills, posted_date])

        print("Job Title : {}".format(job_title))
        print("Company Name : {}".format(company_name))
        print("Rating : {}".format(rating))
        print("Experience : {}".format(ex_wrap))
        print("Location : {}".format(location))
        print("Salary : {}".format(salary))
        print("Minimum Requirements : {}".format(min_requirements))
        print("Skills : {}".format(skills))
        print("Posted Date : {}".format(posted_date))

    print("********PAGE_JOBS END***********")

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

start_page = 1
page_end = 6
for i in range(start_page, page_end + 1):
    print(f"Fetching page: {i}")
    url = generate_url(i, job_role)
    driver.get(url)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "srp-jobtuple-wrapper"))
    )

    sleep(randint(5, 10))  # Allow time for full page load
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')
    page_soup = soup.find_all("div", class_="srp-jobtuple-wrapper")
    parse_job_data_from_soup(page_soup)

driver.quit()
csv_file.close()



#########################################
#  WEBSITE: naukri.com #
######################################### 

Crawl starting time : 18:05:44.744496

Fetching page: 1
********PAGE_JOBS***********
Job Title : Aml Analyst
Company Name : Crisil
Rating : 3.7
Experience : 0-3 Yrs
Location : Bengaluru
Salary : Not disclosed
Minimum Requirements : Bachelor’s degree in finance, law, or business; awareness of AML regulations; strong an...
Skills : amlcddscreeningadvisoryanalyticalfraud preventionanti money launderingconsulting
Posted Date : 2 Days Agosave
Job Title : Urgent Hiring For AML Analyst - Mumbai / Malad !! Salary upto 4 LPA
Company Name : Witbloom Training & Placement
Rating : None
Experience : 1-6 Yrs
Location : Mumbai(Malad West)
Salary : 2.25-4 Lacs PA
Minimum Requirements : Experience- Minimum 1 year on paper exp as a AML Analyst7693928266 1 year of experience...
Skills : AmlAnti Money LaunderingTransaction MonitoringCddSanction ScreeningFraud AnalysisEddRisk Assessments
Posted D